In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os

import march_madness as mm
import random

In [2]:
analysis = mm.Analysis()

In [3]:
analysis.load_season(2019)

In [4]:
analysis.calc_seasons_features()

2019
1104
1380
1113
1168
1119
1265
1120
1375
1123
1232
1130
1454
1131
1318
1138
1384
1144
1423
1148
1216
1151
1150
1155
1154
1158
1342
1165
1127
1166
1442
1172
1156
1176
1263
1181
1246
1187
1175
1189
1264
1197
1239
1199
1196
1207
1271
1209
1190
1211
1226
1221
1357
1229
1195
1231
1152
1235
1106
1242
1277
1250
1284
1256
1455
1260
1282
1261
1368
1266
1420
1268
1174
1269
1262
1272
1399
1276
1313
1278
1303
1281
1146
1283
1331
1301
1291
1304
1290
1305
1140
1307
1169
1308
1295
1312
1159
1314
1459
1323
1227
1330
1298
1332
1340
1333
1415
1334
1188
1335
1206
1338
1464
1344
1373
1345
1193
1348
1136
1355
1213
1359
1466
1360
1451
1361
1115
1362
1413
1364
1461
1371
1447
1376
1367
1378
1105
1385
1259
1387
1369
1390
1370
1392
1203
1393
1186
1396
1247
1400
1183
1403
1230
1409
1108
1411
1124
1412
1273
1414
1225
1416
1351
1417
1236
1419
1238
1422
1299
1425
1352
1429
1286
1433
1205
1435
1457
1437
1288
1438
1406
1444
1178
1449
1443
1458
1164
1462
1237
1112
1223
1137
1382
1173
1316
1177
1126
1267
1184
1326


In [ ]:
season = analysis.seasons[2019]

In [ ]:
season.tourney_seeds

In [ ]:
team = season.teams[1104]

In [ ]:
top64 = season.tourney_seeds
top64

In [ ]:
team.win_data

In [ ]:
top64_wins = team.win_data.merge(top64,left_on='LTeamID',right_index=True)

In [ ]:
for i in top64_wins:
    loc = top64

In [ ]:
top32 = season.tourney_seeds[season.tourney_seeds['Numerical Seed'] <= 8]

In [ ]:
top32

In [ ]:
top32_wins = team.win_data.merge(top32,left_on='WTeamID',right_index=True)

In [ ]:
top32_wins

In [ ]:
weighted_losses = 0
for i in top32_wins.index:
    loc = self.loss_data.loc[i]['WLoc']
                
    weighted_losses += get_loss_location_weight(loc)

In [ ]:
team.win_data

In [ ]:
top16 = season.tourney_seeds[season.tourney_seeds['Numerical Seed'] <= 4]

In [ ]:
top16 = list(top16.index)

In [ ]:
top16

In [ ]:
top8 = season.tourney_seeds[season.tourney_seeds['Numerical Seed'] <= 2]

In [ ]:
top8 = list(top8.index)

In [ ]:
top8

In [5]:
team.win_data

NameError: name 'team' is not defined